# Julia - Day 4

https://adventofcode.com/2021/day/4

## Data

In [63]:
function import_bingo_data(input)
    input = readlines(input)
    draws = parse.(Int, split(input[1], ","))

    n_tables = (length(input) - 1) / 6
    list = [zeros(Int, 5, 5, 2) for _ in 1:n_tables]

    for row in 2:length(input)
        table = (row - 2) ÷ 6 + 1 #tables are 5x5 matrix starting at line 3 with one blank line
        if (row-2) % 6 != 0 #not a blank line
            i = (row-2) % 6
            list[table][i, :, 1] = parse.(Int, split(input[row], r"(?<=\d)\s+", limit = 5))
        end
    end

    return Dict(
        "draws" => draws,
        "bingo_tables" => list
    ) 
end

import_bingo_data (generic function with 1 method)

In [80]:
example = import_bingo_data("Input/4_example.txt")
input = import_bingo_data("Input/4_1.txt")

Dict{String, Vector} with 2 entries:
  "draws"        => [42, 44, 71, 26, 70, 92, 77, 45, 6, 18  …  78, 15, 52, 76, …
  "bingo_tables" => [[48 69 … 49 13; 25 14 … 74 89; … ; 56 97 … 65 79; 57 52 … …

## Part 1

### Function

In [56]:
function draw_position(matrice, draw)
    return findall(isequal(draw), matrice)
end

function update_with_draw!(matrice, draw_pos, value = 1)
    matrice[draw_pos, 2] .= value
end

function matrice_colsum(matrice)
    return [sum(col) for col in eachcol(matrice)]
end

function matrice_rowsum(matrice)
    return [sum(row) for row in eachrow(matrice)]
end

function any_bingo(row_sum_vect, col_sum_vect)
    return any(row_sum_vect .== 5) || any(col_sum_vect .== 5)
end

function sum_not_drawn(matrice)
    return sum(matrice[findall(isequal(0), matrice[:, :, 2]), 1])
end

function bingo(data)

    for draw in data["draws"]

        for i_mat in 1:length(data["bingo_tables"])

            draw_pos = draw_position(data["bingo_tables"][i_mat][:, :, 1], draw)
            update_with_draw!(data["bingo_tables"][i_mat], draw_pos, 1)
            row_sum_vect = matrice_rowsum(data["bingo_tables"][i_mat][:, :, 2])
            col_sum_vect = matrice_colsum(data["bingo_tables"][i_mat][:, :, 2])

            if any_bingo(row_sum_vect, col_sum_vect)
                return sum_not_drawn(data["bingo_tables"][i_mat]) * draw
            end

        end

    end

end
    
    

bingo (generic function with 1 method)

### Result

In [57]:
#Example
result = bingo(example)

println(
    string("Example : ", result)
)
#Input
result = bingo(input)

println(
    string("Input : ", result)
)

Example : 4512
Input : 16674


## Part 2

### Function

In [73]:
function bingo(data)

    bingo_order = zeros(length(data["bingo_tables"]), 2)
    winner_order = 1

    for draw in data["draws"]

        for i_mat in 1:length(data["bingo_tables"])

            draw_pos = draw_position(data["bingo_tables"][i_mat][:, :, 1], draw)
            update_with_draw!(data["bingo_tables"][i_mat], draw_pos, 1)
            row_sum_vect = matrice_rowsum(data["bingo_tables"][i_mat][:, :, 2])
            col_sum_vect = matrice_colsum(data["bingo_tables"][i_mat][:, :, 2])

            if any_bingo(row_sum_vect, col_sum_vect)
                if !(i_mat in bingo_order[:, 1])
                    bingo_order[winner_order, 1] = i_mat
                    bingo_order[winner_order, 2] = sum_not_drawn(data["bingo_tables"][i_mat]) * draw
                    winner_order += 1
                end
            end

        end

    end

    return bingo_order[end, :]

end

bingo (generic function with 1 method)

### Result

In [81]:
#Example
result = bingo(example)

println(
    string("Example : ", result[2])
)
#Input
result = bingo(input)

println(
    string("Input : ", result[2])
)

Example : 1924.0
Input : 7075.0
